1. Extracción y revisión de tdf_tours_master.csv

In [ ]:
import pandas as pd 
import numpy as np
import re

df_tours = pd.read_csv('../data/raw/tdf_tours_master.csv')

display(df_tours)
display(df_tours.info())
display(df_tours.nunique())

2. Limpiamos y transformamos la columna 'Distance'

In [ ]:
#Vemos que la columna 'distance' tiene el formato 2,160.4 km (99.7 mi)
#Eliminamos las millas y los caracteres km y convertimos.

def extract_km (distance_str):
    km_index = distance_str.find ('km')
    km_value = distance_str[:km_index].strip()
    km_value = km_value.replace(',', '')

    return float (km_value)

df_tours['distance_in_km'] = df_tours['Distance'].apply(extract_km)
df_tours.drop(columns = 'Distance', inplace = True)

# display (df_tours)

# Compruebo que no hay valores nulos
# display(df_tours['distance_in_km'].isnull().sum())

#Elimino la columna 'Distance'
display (df_tours)

3. Limpieza de la columna Stages

In [ ]:
#Separo en dos la columna Stages. 
#Dejo dentro de la columna el número de etapas 
#y en una nueva columna el texto

# 1. Limpieza de seguridad: quitamos espacios invisibles (como \xa0) de toda la columna
# Esto es vital para que el número al principio se detecte bien
df_tours['Stages'] = df_tours['Stages'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

# 2. Extracción con una Regex más flexible
# ^(\d+) -> Captura el número inicial
# \s* -> Salta cualquier espacio opcional que haya justo después del número
# (.*)   -> Captura TODO lo que quede hasta el final
df_tours[['Stages_clean', 'stages_observations']] = df_tours['Stages'].str.extract(r'^(\d+)\s*(.*)')

# 3. Mantenemos el número en la columna original y limpiamos la nueva
df_tours['Stages'] = df_tours['Stages_clean']
df_tours['stages_observations'] = df_tours['stages_observations'].str.strip()

# 4. Borramos la columna temporal
df_tours.drop(columns=['Stages_clean'], inplace=True)

#5. Convertimos a null las que no tiene texto en 'stages_observation'
df_tours['stages_observations'] = df_tours['stages_observations'].replace('', np.nan)

# display (df_tours[df_tours['Year'] == 1977])
#Quiero mostrar solo las lineas que no sean nulas en la columna stages_observations
# display (df_tours['stages_observations'].isna().sum())
# display (df_tours['stages_observations'].unique())

#Ahora, según lo que contenga la taba 'stages_observations', se suma a 'Stages' para que de el número correcto de stages.
add_stages = {np.nan: 0, ', including one split stage': 1, ', including six split stages': 6, ', including five split stages': 5, ', including eight split stages': 8,
              ', including one split stages': 1, ', including two split stages': 2, ', including three split stages': 3, '+ Prologue, including two split stages': 3,
              ', including four split stages': 4, '+ Prologue, including three split stages': 4, '+ Prologue, including five split stages': 6,
              '+ Prologue, including four split stages': 5, '+ Prologue, including six split stages': 7, '+ Prologue': 1, '+ Prologue, including one split stage': 2,
              '+ prologue': 1}

extra_stages = df_tours['stages_observations'].map(add_stages).astype(int)
df_tours['Stages'] = df_tours['Stages'].astype(int) + extra_stages

# display(df_tours[df_tours['Year'] == 1977])

4. Creo nueva columna con el ratio de abandonos

In [ ]:
#Creo nueva columna con el % de abandonos
df_tours['abandon_rate'] = (1 - (df_tours['Finishers'] / df_tours ['Starters'])).round(2)
display(df_tours)

5. Limipiamos columna Date

In [ ]:
def normalizar_fechas_final(fecha_texto):
    meses = {
        'January': '01', 'February': '02', 'March': '03', 'April': '04', 
        'May': '05', 'June': '06', 'July': '07', 'August': '08', 
        'September': '09', 'October': '10', 'November': '11', 'December': '12'
    }
    
    texto = str(fecha_texto)
    
    #Limpieza base
    texto = re.sub(r'\[.*?\]', '', texto)          
    texto = re.sub(r'\d{4}$', '', texto)           
    texto = texto.replace('\xa0', ' ')             
    
    # Reemplazamos CUALQUIER carácter que no sea alfanumérico (\w) o espacio (\s) por un '-'
    # Esto cazará automáticamente esos "–" o "—" invisibles y corruptos.
    texto = re.sub(r'[^\w\s]', '-', texto)
    
    # Limpiamos espacios alrededor del nuevo guion y colapsamos espacios dobles
    texto = re.sub(r'\s*-\s*', '-', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()     
    
    # Separación
    partes = texto.split('-')
    
    if len(partes) != 2:
        return fecha_texto 
        
    inicio_str = partes[0].strip()
    fin_str = partes[1].strip()
    
    inicio_partes = inicio_str.split()
    fin_partes = fin_str.split()
    
    try:
        # Lógica A: Mismo mes (ej. "1", "19 July")
        if len(inicio_partes) == 1:
            dia_inicio = inicio_partes[0].zfill(2)
            dia_fin = fin_partes[0].zfill(2)
            mes = meses.get(fin_partes[1], '00')
            return f"De {dia_inicio}/{mes} a {dia_fin}/{mes}"
            
        # Lógica B: Meses distintos (ej. "8 July", "4 August")
        else:
            dia_inicio = inicio_partes[0].zfill(2)
            mes_inicio = meses.get(inicio_partes[1], '00')
            dia_fin = fin_partes[0].zfill(2)
            mes_fin = meses.get(fin_partes[1], '00')
            return f"De {dia_inicio}/{mes_inicio} a {dia_fin}/{mes_fin}"
            
    except Exception as e:
        return fecha_texto

# Aplicar
df_tours['Dates'] = df_tours['Dates'].apply(normalizar_fechas_final)

# Mostrar resultados
display(df_tours[['Year', 'Dates']].head(10))
display(df_tours)

6. Últimos cambios y guardado del DataFrame

In [ ]:
#Ponemos todos los tituls de columnas en minusculas
df_tours.columns = df_tours.columns.str.lower()


#Ordenamos columnas
column_order = ['year', 'dates', 'stages', 'stages_observations', 'distance_in_km', 'starters', 'finishers', 'abandon_rate']
df_clean_tours = df_tours[column_order]

display(df_clean_tours)

df_clean_tours.to_csv('../data/processed/tdf_tours_clean.csv', index=False)